# Phonation Onset Study

## Main Results

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext
import itertools
from pprint import pprint
from IPython.core.debugger import set_trace

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from blockarray import h5utils as bh5utils, blockvec as bv, linalg as bla
from vfsig import modal
from vfsig.misc import resample_over_uniform_period
from femvf.meshutils import process_meshlabel_to_dofs, verts_from_mesh_func

import libhopf
import libsetup
import libsignal
import h5utils
from postprocutils import postprocess

from libsetup import load_transient_model, load_hopf_model

In [ ]:
from main_opt_onsetpressure import (
    ExpParamBasic, ExpParamFreqPenalty, 
    setup_dyna_model
)
from exputils import iter_parameters

In [ ]:
## Set default parameters

EMODS = np.arange(2.5, 20, 2.5) * 1e3 * 10
default_params_basic_dict = {
    'MeshName': 'M5_CB_GA3',
    'Ecov': 2.5e4,
    'Ebod': 2.5e4,
    'ParamOption': 'all',
    'Functional': 'OnsetPressure'
} 
default_params_penalty_dict = {
    'MeshName': 'M5_CB_GA3',
    'Ecov': 2.5e4,
    'Ebod': 2.5e4,
    'ParamOption': 'all',
    'Functional': {
        'Name': 'OnsetPressure',
        'omega': -1,
        'beta': 1000
    }
} 

DEFAULT_PARAMS_BASIC = ExpParamBasic(default_params_basic_dict)
DEFAULT_PARAMS_PENALTY = ExpParamFreqPenalty(default_params_penalty_dict)

In [ ]:
## Functions for loading sensitivty/minimization simulation results

def load_sensitivity_scalars(
        substitute_params,
        dset_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        n=0
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    props = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'out/sensitivity/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            props.append(f[dset_name][:])
    return props

def load_sensitivity_vectors(
        substitute_params,
        group_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        n=0
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    props = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'out/sensitivity/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            
            if isinstance(n, (list, np.ndarray, tuple)):
                vecs = [
                    bh5utils.read_block_vector_from_group(f[group_name], nvec=_n)
                    for _n in n
                ]
            else:
                vecs = bh5utils.read_block_vector_from_group(f[group_name], nvec=n)
            props.append(vecs)
            # print(vecs)
    return props

def load_minimization_vectors(
        substitute_params,
        group_name, 
        it=-1,
        default_params=DEFAULT_PARAMS_PENALTY
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    props = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'out/minimization/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            props.append(bh5utils.read_block_vector_from_group(f[group_name], nvec=it))
    return props

HOPF_MODEL, *_ = setup_dyna_model(DEFAULT_PARAMS_BASIC)
MESH = HOPF_MODEL.res.solid.forms['mesh.mesh']
VEC_CG1_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.vector'].dofmap()
CG1_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.scalar'].dofmap()
DG0_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.scalar_dg0'].dofmap()
CELL_TO_SDOF_CG1 = CG1_DOFMAP.entity_closure_dofs(MESH, 2)
CELL_TO_SDOF = DG0_DOFMAP.entity_closure_dofs(MESH, 2)

In [ ]:
# Get a list of vertices on the medial region
forms = HOPF_MODEL.res.solid.forms

facet_func = forms['mesh.facet_function']
facet_label_to_id = forms['mesh.facet_label_to_id']
print(facet_label_to_id)
VERTS_MED = verts_from_mesh_func(MESH, facet_func, 3)
VERTS_DIR = verts_from_mesh_func(MESH, facet_func, 4)
# VERTS_MED = 
# TSHAPE_PARAM = 
VERT_TO_VDOF = dfn.vertex_to_dof_map(HOPF_MODEL.res.solid.forms['fspace.vector'])

### Plot configuration

In [ ]:
## Plot style
FIG_STYLE = 'presentation'

if FIG_STYLE == 'presentation':
    RCPARAMS = {
        'font.family': 'sans-serif',
        'font.size': 24
    }
elif FIG_STYLE == 'manuscript':
    RCPARAMS = {
        'font.family': 'sans-serif',
        'font.size': 10
    }
mpl.rcParams.update(RCPARAMS)

In [ ]:
def plot_tripcolor_sequence(
        fig, axs, coordss, cells, zs,
        plot_type='tripcolor',
        **plot_kwargs
    ):
    zmin = np.nanmin([np.nanmin(z) for z in zs])
    zmax = np.nanmax([np.nanmax(z) for z in zs])
    print(zmin, zmax)
    
    if plot_type == 'tripcolor':
        artists = [
            ax.tripcolor(*coords.T, cells, z, vmin=zmin, vmax=zmax, **plot_kwargs)
            for ax, z, coords in zip(axs, zs, coordss)
        ]
    elif plot_type == 'tricontourf':
        artists = [
            ax.tricontourf(*coords.T, cells, z, vmin=zmin, vmax=zmax, **plot_kwargs)
            for ax, z, coords in zip(axs, zs, coordss)
        ]
    else:
        raise ValueError(f"Unknown 'plot_type' '{plot_type}'")
    return artists

### Sensitivity Results


In [ ]:
params = DEFAULT_PARAMS_BASIC
fpath = f'out/sensitivity/{params.to_str()}.h5'
with h5py.File(fpath, mode='r') as f:
    print(f.keys())
    print(f['eigvals'])

In [ ]:
emods = EMODS

# functional_name = 'OnsetFrequency'
functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'
functional_name_to_symb = {
    'OnsetFrequency': 'f_\mathrm{onset}',
    'OnsetPressure': 'p_\mathrm{onset}',
    'OnsetPressureStrainEnergy': 'G'
}
functional_symb = functional_name_to_symb[functional_name]

props_list = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': functional_name}, 
        'props'
    )[0] 
    for emod in emods
]

## Load first/ second order sensitivity
dfon_dparams = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetFrequency'}, 
        'grad_param'
    )[0] 
    for emod in emods
]

dpon_dparams = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetPressure'}, 
        'grad_param'
    )[0] 
    for emod in emods
]

# test = load_sensitivity_vectors(
#     {'Ecov': 2.5e4, 'Ebod': 2.5e4, 'Functional': 'OnsetFrequency'}, 
#     'hess_props', n=np.arange(5)
# )
# print(len(test))

d2fon_dparams2 = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetFrequency', 'ParamOption': 'const_shape'}, 
        'hess_props', n=n
    )[0]
    for n in np.arange(5)
    for emod in emods
]

eigval_d2fon_dparams2 = [
    load_sensitivity_scalars(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetFrequency', 'ParamOption': 'const_shape'}, 
        'eigvals'
    )[0][n]
    for n in np.arange(5)
    for emod in emods
]

d2pon_dparams2 = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetPressure', 'ParamOption': 'const_shape'}, 
        'hess_props', n=n
    )[0]
    for n in np.arange(5)
    for emod in emods
]

eigval_d2pon_dparams2 = [
    load_sensitivity_scalars(
        {'Ecov': emod, 'Ebod': emod, 'Functional': 'OnsetPressure', 'ParamOption': 'const_shape'}, 
        'eigvals'
    )[0][n]
    for n in np.arange(5)
    for emod in emods
]
# print(len(d2pon_dparams2[0]))

# projected gradient of p
dppon = [
    dpon_dparam - bla.dot(dpon_dparam, dfon_dparam)*dpon_dparam
    for dpon_dparam, dfon_dparam in zip(dpon_dparams, dfon_dparams)
]

# dprops_list = [
#     load_sensitivity_vectors(
#         {'Ecov': emod, 'Ebod': emod, 'Functional': functional_name}, 
#         'dprops'
#     )[0] 
#     for emod in emods
# ]
dparams_list = [
    load_sensitivity_vectors(
        {'Ecov': emod, 'Ebod': emod, 'Functional': functional_name}, 
        'grad_param'
    )[0]
    for emod in emods
]

props_list[0].print_summary()

dparams_list = dppon

In [ ]:
CASE = 'OnsetPressure'

In [ ]:
if CASE == 'OnsetPressure':
    params_lists = [dpon_dparams]
    symbols = [r"-dp_\mathrm{on}/d\mathbf{E}"]
elif CASE == 'OnsetFrequency':
    params_lists = [dfon_dparams]
    symbols = [r"-df_\mathrm{on}/d\mathbf{E}"]
else:
    params_lists = [
        dfon_dparams, 
        dpon_dparams, 
        dppon
    ]
    symbols = [
        r"-df_\mathrm{on}/d\mathbf{E}",
        r"-dp_\mathrm{on}/d\mathbf{E}",
        r"-d\hat{p}_\mathrm{on}/d\mathbf{E}"
    ]
N = len(symbols)
    
    
fig = plt.figure(figsize=(30, 13))
gs = mpl.gridspec.GridSpec(2*N, len(emods), figure=fig, height_ratios=[0.05, 0.95]*N)

axs = np.array([[fig.add_subplot(gs[2*i+1, j]) for j in range(gs.ncols)] for i in range(gs.nrows//2)])
axs_cbar = np.array([fig.add_subplot(gs[2*i, 0:3]) for i in range(gs.nrows//2)])
# print(axs)
# axs = np.atleast_1d(axs)

coords = MESH.coordinates()
cells = MESH.cells()

#### Plot sensitivity for each case
for ii, (dparams_list, symb) in enumerate(zip(params_lists, symbols)):
    ## Plot emod sensitivity
    zs = [-dparams['emod'][CELL_TO_SDOF] for dparams in dparams_list]
    artists = plot_tripcolor_sequence(fig, axs[ii, :], len(zs)*[coords], cells, zs)
    
    fig.colorbar(artists[0], cax=axs_cbar[ii], orientation='horizontal')
    axs_cbar[ii].set_xlabel(f"${symb}$")
    axs_cbar[ii].tick_params(labelrotation=25)

    ## Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    ax.set_ylim(0, 0.6)

for ax_row in axs:
    for ax, emod in zip(ax_row, emods):
        ax.text(0.05, 1.05, f"{emod/10/1e3:.1f} kPa", transform=ax.transAxes)
    
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[:, 1:].flat:
    ax.tick_params('y', labelleft=False)

fig.tight_layout()
fig.savefig(f'Gradient_{CASE}.svg')

In [ ]:
fig = plt.figure(figsize=(30, 30))

neig = 5
gs = mpl.gridspec.GridSpec(1+neig, len(emods), figure=fig, height_ratios=[0.05]+[1]*neig)

axs = np.array([[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)])
ax_cbar = fig.add_subplot(gs[0, 0:3])
# print(axs)
# axs = np.atleast_1d(axs)

coords = MESH.coordinates()
cells = MESH.cells()

symb = "Eigenvector"

if CASE == 'OnsetPressure':
    dparams = d2pon_dparams2
    eigvals = eigval_d2pon_dparams2
elif CASE == 'OnsetFrequency':
    dparams = d2fon_dparams2
    eigvals = eigval_d2fon_dparams2

# dparams = d2fon_dparams2
# eigvals = eigval_d2fon_dparams2
        
## Plot emod sensitivity
zs = [eigvec['emod'][CELL_TO_SDOF] for eigvec in dparams]
zs = zs[:neig*len(emods)]

artists = plot_tripcolor_sequence(
    fig, axs.reshape(-1), len(zs)*[coords], cells, 
    zs
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(f"${symb}$")
ax_cbar.tick_params(labelrotation=25)

## Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_ylim(0, 0.6)
    
for ax, eigval in zip(axs.flat, eigvals):
    ax.text(
        0.05, 0.95, f"$\lambda=$ {eigval:.1f}", 
        transform=ax.transAxes, va='top'
    )

for ax, emod in zip(axs[0, :], emods):
    ax.text(0.05, 1.05, f"{emod/10/1e3:.1f} kPa", transform=ax.transAxes)

for ax in axs[-1, :]:
    ax.set_xlabel("x [cm]")
for ax in axs[:, 0]:
    ax.set_ylabel("y [cm]")

for ax in axs[:, 1:].flat:
    ax.tick_params('y', labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.tick_params('x', labelbottom=False)

fig.tight_layout()

fig.savefig(f'Hessian_{CASE}_{neig}.svg')

### Nonlinear Minimization Results


In [ ]:
emods = EMODS

# functional_name = 'OnsetFrequency'
functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'

param_option = 'const_shape'
# param_option = 'all'

it = -1
hopf_states = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_state', it
    )[0] 
    for emod in emods
]

props_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_props', it
    )[0] 
    for emod in emods
]

params_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'parameters', it
    )[0] 
    for emod in emods
]

dprops_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'grad', it
    )[0] 
    for emod in emods
]

omegas = [
    np.abs(load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_state', 0
    )[0]['omega'][0])
    for emod in emods
]

emod = props_list[0].sub['emod']
emod.min() - emod.max()

In [ ]:
fig = plt.figure(figsize=(15, 5))
gs = mpl.gridspec.GridSpec(2, len(emods), figure=fig, height_ratios=[0.05, 0.95])
axs = np.array([fig.add_subplot(gs[-1, i]) for i in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, 0:3])

def get_coords(props, state):
    HOPF_MODEL.set_props(props)
    ufp = state['u']
    ufp_vert = np.array(ufp[VERT_TO_VDOF]).reshape(-1, 2)
    
    coords = HOPF_MODEL.res.solid.forms['mesh.mesh'].coordinates() + ufp_vert
    return coords.copy()

coordss = [get_coords(props, state) for props, state in zip(props_list, hopf_states)]
coords = coordss[-1]
cells = MESH.cells()
print([np.linalg.norm(coords) for coords in coordss])

_fspace_cg1 = HOPF_MODEL.res.solid.forms['fspace.scalar']
_fspace_dg0 = HOPF_MODEL.res.solid.forms['fspace.scalar_dg0']
_function_dg0 = dfn.Function(_fspace_dg0)
_function_cg1 = dfn.Function(_fspace_cg1)
def project_dg0_to_cg1(vec):
    _function_dg0.vector()[:] = vec
    # _function_cg1.vector()[:] = vec
    dfn.project(_function_dg0, V=_fspace_cg1, function=_function_cg1)
    return np.array(_function_cg1.vector()[:])

VERT_TO_SDOF = dfn.vertex_to_dof_map(_fspace_cg1)
zs = [
    project_dg0_to_cg1(props['emod'])[VERT_TO_SDOF]/10/1e3 
    for props in props_list
]
# print(zs[0])
artists = plot_tripcolor_sequence(
    fig, axs, coordss, cells, zs, 
    plot_type='tricontourf', extend='neither', levels=np.linspace(0, 30, 11)
)
# print(artists[0])
# artists[0] + artists[1]
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$E$ [kPa]")
# ax_cbar.set_xlabel(r"$\frac{dg}{d\mathbf{E}}$")

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    ax.set_ylim(0, 0.52)
    ax.set_xlim(0, 0.8)
    
for ax, omega in zip(axs, omegas):
    ax.text(0, 1.05, f"$f_0$={omega/2/np.pi:.1f} Hz", transform=ax.transAxes)
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[1:]:
    ax.tick_params('y', labelleft=False)

fig.tight_layout()

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
# PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
PSUBS = np.arange(550, 650, 10) * 10 
ECOV = 5e3 * 10
# EBODY = 15e3 * 10
EBODY = 5e3 * 10

# Load the transient model
MESH_NAME = 'BC-dcov5.00e-02-cl1.00'
MESH_NAME = 'M5_CB_GA3'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

# print(model_tran.solid.forms['mesh.vertex_label_to_id'])
# print(model_tran.solid.forms['mesh.facet_label_to_id'])
# print(model_tran.solid.forms['mesh.cell_label_to_id'])

In [ ]:
## Initialize functions to extract glottal width from saved simulations

# Number of points per period to use in processing
N_PER_PERIOD = 100
proc_gw_tran = make_sig_glottal_width_sharp(model_tran)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Load mesh and DOF map information for plotting data on the mesh
TRI = res.solid.forms['mesh.mesh'].coordinates()
X, Y = TRI[:, 0], TRI[:, 1]
CELLS = res.solid.forms['mesh.mesh'].cells()

# These indices select DOFs from a scalar function in vertex-order
IDX_VERT = dfn.vertex_to_dof_map(res.solid.forms['fspace.scalar'])

XREF = res.solid.XREF.vector()
VERT_TO_VDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])

# Conversion for centimeter
CM = 1/2.54

## Miscellaneous Experiments

### Growth Rate Plot for Generic Model

Plots of the growth rate vs subglottal pressure.

In [ ]:
MESH_NAME = 'M5_CB_GA1'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

In [ ]:
for key, subvec in res.props.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")
    
for key, subvec in res.control.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

In [ ]:
psubs = np.arange(0, 800, 20)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 5), sharex=True)

axs[0].plot(psubs, modes.real)
axs[1].plot(psubs, modes.imag)

axs[0].axhline(0, color='k', ls='-.')

axs[0].set_ylabel("$\omega_{real}$")
axs[1].set_ylabel("$\omega_{imag}$")
axs[1].set_xlabel("$p_{sub}$")

# axs[0].legend()
# axs[1].legend()

fig.tight_layout()

### Bifurcation plots for smooth minimum separation fluid models

In [ ]:
# Set the model properties and sanity check the values
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar']
)
props = libsetup.set_props(res.props, _region_to_dofs, res)

props['zeta_min'] = 1e-8
props['zeta_sep'] = 1e-4
res.set_props(props)

# proplabel_to_norm = {label: subvec.norm() for label, subvec in props.items()}
# pprint(proplabel_to_norm)

proplabel_to_max = {label: subvec.max() for label, subvec in props.items()}
pprint(proplabel_to_max)
print(res.solid.forms['mesh.mesh'].coordinates()[..., 1].max())

#### Plot the growth rate vs $p_{sub}$

In [ ]:
zeta_min = 1e-6
zeta_sep = 1e-2
props = res.props
props['zeta_min'] = zeta_min
props['zeta_sep'] = zeta_sep
res.set_props(props)

# psubs = np.arange(585, 590, 0.1)*10
psubs = np.arange(700, 800, 10)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
def jac_block_norms(res, psub):
    xfp, info = libhopf.solve_fp(res, psub)
    res.set_state(xfp)
    res.control['psub'].array[0] = psub
    res.set_control(res.control)
    
    dres_dstate = res.assem_dres_dstate()
    return tuple([submat.norm() for submat in dres_dstate.subarrays_flat])

block_norms = [jac_block_norms(res, psub) for psub in tqdm(psubs)]
block_labels = [','.join(multi_label) for multi_label in itertools.product(res.state.labels[0], res.state.labels[0])]

block_norms = np.array(block_norms)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True)
    
ii_max = np.argmax(modes.real)
print(psubs[ii_max])

axs[0].plot(psubs, modes.real)
axs[0].axhline(0, color='k', ls='-.')
axs[0].set_ylabel("$\omega_{real}$")

axs[1].plot(psubs, modes.imag)
axs[1].set_ylabel("$\omega_{imag}$")

for norms, label in zip(block_norms.T, block_labels):
    axs[2].plot(psubs, norms, label=label)
axs[2].set_ylabel("$|| \\frac{dF}{dx} ||$")
axs[2].set_xlabel("$p_{sub}$")
# axs[2].set_yscale('log')
axs[2].legend()

# axs[0].set_ylim(-10, 50)

fig.savefig(f'fig/GrowthRatevsPsub_zetamin{zeta_min:.2e}_zetasep{zeta_sep:.2e}_closeup.png', dpi=200)

### Strange bifurcation behaviour for smoothed minimum separation models

Rapid changes in eigenvalues can occur with small changes in $p_{sub}$ when using a smooth minimum approximation. This happens when the smooth minimum approximation approaches the true minimum where small changes in areas rapidly shift weights between nodes.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)

s = np.linspace(0, 1, 12)
a = (s-0.5)**2 + 0.1
da = 1e-2*s

def smoothmin(a, zeta=1e-4):
    log_w = -a/zeta
    log_w = log_w - log_w.max()
    print(log_w)
    _w = np.exp(log_w)
    print(_w.max())
    return _w/np.sum(_w)

axs[0, 0].plot(s, a, marker='.')
axs[0, 0].set_title("a [cm]")
axs[1, 0].plot(s, smoothmin(a))
axs[1, 0].set_ylim(0, 1)

axs[0, 0].set_ylabel("Area [cm]")
axs[1, 0].set_ylabel("smoothmin weight")

axs[0, 1].plot(s, a+da, marker='.')
axs[0, 1].set_title("a + da [cm]")
axs[1, 1].plot(s, smoothmin(a+da))
axs[1, 1].set_ylim(0, 1)

# axs[0, 0].yaxis.
for ax_row in axs:
    for col in range(1, ax_row.size):
        ax_row[0].sharey(ax_row[col])
        ax_row[col].yaxis.set_tick_params(labelleft=False)

## Debugging

For the case where we minimize onset pressure while maintaining a constant onset frequency, the optimization encounters a problem. This is illustrated below for the history file 'OPT_DEBUG.h5' which started from a uniform initial guess of 5 kPa.

At some of the last iterations, optimization progress stalls and the optimization routine fails with a zero size step on a line search. A likely reason why this occurs can be seen by plotting the maximum real eigenvalue as a function of subglottal pressure. The minimal subglottal pressure of 0.38 kPa occurs due to a very sharp peak in the profile. It is likely that as parameters change in the line search, this small peak disappears so that onset jumps to the next onset pressure of around 0.5 kPa.

In [ ]:
# Load optimization history
opt_fname = 'OPT_DEBUG_1'
with h5py.File(f'out/{opt_fname}.h5', mode='r') as f:
    objs, grads, params, xhopfs = load_opt_hist(f)
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([xhopf['psub'][0] for xhopf in xhopfs])
onset_fs = np.array([xhopf['omega'][0] for xhopf in xhopfs])

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(its, objs)
axs[0].set_ylabel("Objective")

axs[1].plot(its, onset_ps)
axs[1].set_ylabel("Onset pressure")

axs[2].plot(its, np.abs(onset_fs))
axs[2].set_ylabel("Onset freq.")

axs[2].set_xlabel("")

### Compute a line search along a given direction

In [ ]:
# compute functionals along the line from N -> N+1
N = 25
res_hopf.set_state(bvec.convert_subtype_to_petsc(xhopfs[N]))
res_hopf.set_props(bvec.convert_subtype_to_petsc(params[N]))

xhopf_n, info = libhopf.solve_hopf_newton(res_hopf, res_hopf.state)
print(info)
print(xhopf_n['psub'][0])

### Compute the spectrum at a specific iteration

In [ ]:
# Specify the parameter set from the optimization to use
N = 26
res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))

In [ ]:
## Plot the spectrum at the onset pressure

def plot_spectrum(fig, ax, xhopf):
    res.set_props(bvec.convert_subtype_to_petsc(params[N][:-2]))
    res.set_control(res.control)

    xfp_n = xhopf[res_hopf.labels_fp]
    psub = xhopf['psub'][0]
    omegas, eigvecs_real, eigvecs_imag = libhopf.solve_modal(res, xfp_n, psub)
    
    ax.scatter(omegas.real, omegas.imag)
    ax.axhline(0, color='k')
    ax.axvline(0, color='k')

    ax.set_xlabel("$\Re(\omega)$")
    ax.set_ylabel("$\Im(\omega)$")
    ax.set_title(f"$P_\mathrm{{onset}}$={onset_ps[N]/10:.1f} Pa")
    ax.set_xlim(-50, 10)
    ax.set_ylim(-1200, 1200)

    fig.tight_layout()
    return fig, ax

# for n in range(0, 27):
#     fig, ax = plt.subplots(1, 1)
#     fig, ax = plot_spectrum(fig, ax, n)
#     fig.savefig(f"fig/Spectrum{n}.png")
#     plt.close(fig)
    
fig, ax = plt.subplots(1, 1)
fig, ax = plot_spectrum(fig, ax, xhopfs[N])

In [ ]:
## Compute the most unstable eigenvalue over a range of subglottal pressures
# This gives a rough idea at which subglottal pressure(s) (if any) onset will occur

# psubs = np.arange(300, 600, 2.5)*10
psubs = np.arange(0, 2600, 100)*10
omegas_max = np.zeros(psubs.shape, dtype=complex)

least_stable_modes_info = [libhopf.solve_least_stable_mode(res_hopf.res, psub) for psub in psubs]
omegas_max = np.array([least_stable_info[0] for least_stable_info in least_stable_modes_info])

# xfp_0 = res.state.copy()
# xfp_0.set(0.0)

# for ii, psub in enumerate(psubs):
#     res.control['psub'][0] = psub
#     res.set_control(res.control)

#     xfp_n, info = libhopf.solve_fixed_point_newton(res, xfp_0, newton_params={'max_iter': 20})
#     # print(f"Solving for fixed point took {info['num_iter']} iterations. Abs err {info['abs_errs'][-1]}")
#     omegas, eigvecs_real, eigvecs_imag = libhopf.solve_linear_stability(res, xfp_n)
    
#     idx_max = np.argmax(omegas.real)
#     omegas_max[ii] = omegas[idx_max]

In [ ]:
print(np.array(omegas_max).shape)
print(psubs.shape)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), sharex=True)

# idx = np.logical_and(psubs >= 2000, psubs <= 6000)
idx = np.ones(psubs.shape, dtype=bool)

axs[0].plot(psubs[idx], np.array(omegas_max)[idx].real)
axs[0].axhline(0, color='k')
axs[0].set_ylabel("$real \; \omega$ $[rad/s]$")


axs[1].plot(psubs[idx], np.array(omegas_max)[idx].imag)
axs[1].axhline(0, color='k')
axs[1].set_ylabel("$imag \; \omega$ $[rad/s]$")

# axs[1].set_xlim(3500, 4000)5
# axs[0].set_ylim()


axs[1].set_xlabel("$P_{sub}$ $[Pa]$")